# Relevance in Deeplinked Embedded Ads for URX

The goal of this project is to construct a model that can effectively predict the relevance between a publisher's webpage and an advertiser's webpage.

## Setup:

### Background: 

There are two general strategies that advertisers can employ to capture potential customers' attention. The first strategy is to find the apppropriate context in which the customer will be most amemnable to receiving the advertisement. This is putting a billboard advertising bottled water in the middle of the desert. The second strategy is to make an advertisement which is sufficiently entertaining that your customer won't mind (or perhaps even realize) that the ad isn't relevant to the context. This is putting a billboard with a hilarious jokes and lots of nudity promoting a new movie in the desert. 

[put in the billboard graphic from the powerpoint slide]

In the twentieth century, as we developed broadcast-based, mass media, the same opportunities that allow brands to advertise to massive audiences do not allow them to target the audience when they're most ammenable to their products' value propositions. Most people are not actively thinking about a new car while watching their favorite television program, and this fact of circumstance has forced the advertising industry to focus on creative over relevance. 

With the Internet and increasingly sophisticated targeting, demographic, and context information, relevance is now an increasingly effective strategy for advertising. 

URX is an advertising technology startup which has focused almost exclusively on advertising by context: all of their ads look the same. 

### Two challenges:

Revise: two challenges: 1. it is subjective 2. it is not directly observable

Relevance is subjective and depends on the beliefs and information state of the customer. Even when in the fairly obvious case of the man crawling through the desert, bottled water might seem like an obviously relevant ad, but it might be less relevant than a subscreen ad. That is: even in what might seem like the most obvious situations, relevance is subjective. The second challenge is that relevance is not directly observable. You can observe views, loads, clicks, actions, and a variety of other measureable metrics in online marketing, but not whether an ad was relevant. Of course, more relevant ads should be clicked more frequently, but there is no way to determine whether the higher click rate is a matter of relevance, ad copy, picture, or even just the novelty effect that comes with displaying anything new. 

### Two approximations:

URX has decided to handle the challenges of relevance in two ways. The first is that they've defined relevance with great precision in order to improve intersubjective agreement. Two pages are relevant if the publisher's page is about the same named entity as the advertiser's page. A named entity is essentially a proper noun: Michael Jackson, Goldman Sachs, the Stanley Cup. They've decided to overcome the fact that relevance is not directly observable by using a crowdsourced human intelligence platform, Crowd Flower. Crowd Flower is a platform that allows you to offer small tasks to be completed by humans, and in return for a few cents, people will tell you which of the following pictures contains spagetti, what the most recent price of an item is on Amazon, or whether two pages are about the same named entities. For each publisher/advertiser pair, 

### Experimental Design

Ben Bowles at URX designed and ran an experiemnt with Crowd Flower. Because URX has two different algorithms to select a relevant ad, for each publisher URL, ther eare two different advertiser URL's. In total, the experiment contains 950 distinct publisher/advertiser pairs. For each pair, either 8 or 10 workers were asked to rank the relevance of the publisher/advertiser webpages on a scale of 1 to 3. 1 indicates that the two are irrelvant, that is, they are not about the same named entities. 3 indicates that they are both about the same named entities. 2 indicates that the pages do not contain the same named entities, but there is a named entitity connection that can be made, for example, one page is about Kurt Cobain and the other about Nirvana, and since Cobain is the singer of Nirvana, there is a named entity connection, but it is not obvious. Some workers might have used 2 to indicate when they were confused. I normalized all scores to be between 0 and 1, and will refer to the normalized scores throughout this report Because of quality control issues that I will discuss right now, a second experiment was run, using the same publisher/advertiser url pairs, but with tighter quality controls. Throughout the rest of this report, I'll show the results from the first and second experiments next to each other in order to demonstrate the improvement in quality that came out of the insights from the first experiment into the second one. 


### Essential insights to look out for throughout this report 

This project has a couple of insightful themes. 
1. The importance of clear definitions and using language very carefully. 
2. The importance of quality control in crowd sourced data. 
3. The importance of evaluting results using metrics of specific interest to the busines. 




# Overview of the project pipeline 

There are essentially three stages in my pipeline: 

1. Preprocessing the data 
        1. Detecting fraud
        2. Aggregating the relevance scores of different workers
        3. Extracting and engineering features from the publisher and advertiser webpage 
        4. Reduction in data set size 
2. Training the model
        A. Choosing the model
3. Validating the model 
        A. Performance metrics 
        B. Business metrics 
4. Pipeline

I will now, in painful detail, go through each of the stages and substages of my pipeline. If you fall asleep easily, or don't care about details, or perhaps most foolishly of all, trust me to handle the details correctly, you can skip ahead to the next section in which I get to the results and the insights. 

## 1. Preprocessing the data 

In order to avoid garbage in, garbage out, the input data needs to be preprocessed and checked for quality. There were two problems with the quality of the scores that the workers provided. The first is that some of the scores were fraudulant. The second is that there was little consensus around relevance. I'll address these two in that order.

### 1.1 Detecting Fraud:

Fraud detection is a well-established problem in data science. Supervised learning approaches require a labeled data set in order to train a prediction algorithm. While my data clearly contained fraud, I did not have ground truth labeled data to train a model with. This leaves me with an unsupervised learning task. There are essentially two appraoches in a situation like this: clustering and anomaly detection. In clustering, you attempt to suss out groups (fraud and not-fraud) inside some feature space of the observations. In anomaly detection, you establish some 'normal' range that you observations ought to fall into, and then identify those which fall above or below the normal thresholds. Before applying either of these approaches, however, you need to have some kind of feature space. 


##### Feature space

I started by grouping the data set according to workers rather than  publisher/advertiser url pairs and came up with five different features for workers that could be used to detect anomalous/fraudulent behavior. 

    1. Job count per worker

As your intution would indicate, the distribution of jobs per worker looks approximately log normal. That is, there is a concerntrated lump, and a long right tail. Again, consistent with intution, fraudulent workers take on more jobs. So the observed distribution of jobs per worker looks like a log normal distribution with a big spike toward the end where a small number of workers (but much more than you would expect) took on an enormous number of jobs. 

[job count histrogram from experiment 1]
[job count histogram from experiment 2]


    2. Score variance per worker

Assuming that workers were given jobs randomly, the distribution of scores per worker should be, with a margin of error, approximately the same as that of publisher/advertiser relevance scores as a whole. So you would expect the variance of different workers to be approximately normally distributed. Some workers, however, had a variance of zero. That is, they entered (typically 0) the same score for every single job they did.

[worker variance exp1]
[worker variance exp2]


    3. Mean score per worker

Similarly to the score variance per worker, given the global mean and the number of jobs a worker takes, we can establish a 95% confidence ratio for the mean score per worker. 

[mean score per worker exp1]
[mean score per worker exp2]

[bar chart: x-axis: number of jobs per worker, y axis mean, with points showing the narrowing window for means, and then worker mean scores plotted on top]

    4. Time stamp difference regularity

The Crowd Flower csvs contain timestamp information about the datetime for the creation of each job, and for when the worker actually started the job. The distribution of the difference between these timestamps should be normally distributed around the average amount of time it takes the worker to get started. However, when you look at the actual histograms, some workers have a nice normal distribution, but others have a strinking regularity to how long it takes them to start the job, indicative of an automated script (bot). 

[example of normally distributed time difference]
[exmaple of overly regular distributed time difference]

However, this regularity isn't going to be captured by a simple measure on the distribution of timestamp differences such as the mean or the variance. In order to capture this, I took the number of unique timestamp difference values over the total number of jobs that the worker did. We can see that this measure, looking across all of the workers, is approximately normal, with the suspiciously regular timestamp differences falling outside of the 95% confidence interval. 

[histogram of timestamp differences for exp1]
[histogram of timestamp differences for exp2]

    5. Frequency of minority judgments 

Worker's aren't always going to be correct. Let's call a judgment correct if it falls into the majority and wrong if it falls into the minority. Unlike the other features which I aggregated at the worker level, this one I aggregated at the country level. The thinking here is that a worker might be in the minority on too many votes by one of three mechanisms: either the worker is a mindles robot, the worker is a thoughtless fraud, or the worker is just confused. Since the instructions and website are all in English, I want the feature to account for the last possibility. The distribution of errors, by country or by worker, are both roughly normal, but some people fall outside of the 95% confidence interval. 

[histogram of minority judgments by country exp1]
[histogram of minority judgments by country exp2]

As should be pretty obvious, we can't account for the minority judgments as a result of confusion when one of the highest rates of minority judgmnets is the United States. Moreover, suppose that you elimate fraud and error using the first four judgments. If you then, afterwards, remove what else remains using the country metric, this changes the size of the data set by less than 2%

[put little table with the post country size change for exp 1]
[put little table with the post country size change for exp 2]


##### Aggregating the fraud features

There are two advantages to using multiple measures to catch fraud. In the short term the advantage is that, since this is an unsupervised classification task, there are multiple, somewhat independent information sources that we can use to triangulate the frauds. This is important becuase some poeple do many tasks, but are not fraudulent, or becuase some bots/frauds have a minimum threshold for variance, but they still end up producing means that outside of the confidence interval. The second advantage is that fraud is typically a long term, adversarial relationship between fraudsters and whowever it is that they're trying to defraud. If you block one avenue, they'll get around that one, but if you have five measures, they can't defeat them all at once, and perhaps more critically, if they defeat one, you still have four more features to infer fraud with. In an ideal world, each of the measures of fraud would be somewhat correlated with the other features, but imperfectly so becuase they each catch a distinct although related aspect of the fradulent activity. 

[scatterplots of each fraud feature against each other exp1]
[scatterplots of each fraud feature against each other exp2]

There are three ways to aggregate the fraud features. The most sophisticated approach is to ensure that each one represent a probability, establish independence between each feature, and then multiply them for a conditional probability of fraud given all five features. The problem with this is that they are not independent, and and the different measures are not probabilities. The second appraoch, which I implemented, is to apply a threshold for each to the measure so that each has a binary output, add them all up, and then apply a rule-of-thumb threshold. This is what I did, using basic anomaly detection for each threshold individually, and then setting the rule that if a worker violated three or more measures, it was classified as fraud. 

[table of summary statistics for how many were rejected exp1]
[table of summary statistics for how many were rejected exp2] 

It should be no surprise that roughly 10% [adjust later] of the workers were fraudulent, and that these accounted for nearly 20% of the jobs. It should be noted that the groud truth as to whether this actually identifies bots or not is irrelevant; these workers produce judgments that are of sufficiently low quality that, robots or not, we do not want to include them in our experiemnts. 

The second way to aggregate the fraud measures is by using the worker/fraud matrix to cluster the workers into two groups. I did not have time to do this, but if I did, I would then have put effort into determining the difference between its output and the output of the previous, simpler approach. 


### 1.2 Aggregating the relevance scores of workers: 

There are two general approaches to aggregating the relevance score, each of which reflects a different underlying concept of relevance. You can take the average of the votes, and end up with some value between 0 and 1, inclusive, or you can have a winner-take-all vote system in which the relevance value for each publisher/ad pair is either 0 or 1. The first method looks at relevance as the term is used in normal, everyday English. The problems with this are twofold: this definition of relevance is highly subjective, and it contradicts the precise definition (being about the same named entity) that URX is using. This basically leaves us with the second approach, however there are two general approaches to aggregating the individual worker judgments into a binary. The first is to go with a majority vote (perhaps throwing out the publisher/advertiser pairs for which there is not a high enough consensus), and the second is to take the mean and then round up or down contingent on some threshold. I opted for the second approach, not dissimilar to the electoral college system. Before going into the details here, I want to present the histogram of the normalized mean for the first and second experiments. 

[histogram of normalized mean exp1]
[histogram of normalized mean exp2]

You'll notice that there is a lot of consensus around what is not irrelevant, but once things are relevant, the subjective starts to become a serious problem. This is not a bimodal distribution, or a hurdle or zero-inflated distribution: this is noise. It's not entirely meaningless, and I still managed to train a decent model, but there was too much noise to noise to create a powerful predictive model. Part of why I created so many measures for fraud was in the hope that I could remove enough of the problematic workers to improve the distribution. This is also why there were two experiments: after trying a variety of approaches to clean the data, engineer the features, and select and tune the models, it was clear that the model needed to be run again. The way that Crowd Flower enforces quality control is by having a set of quality control questions for which the answer is known, and interspersing those among the questions you want the workers to answer. If the workers pass the minimum threshold, they get to continue asnwering questions and get paid for their work, if they fail their responses are thrown out, as are they. In the first experiment the quality threshold, called "trust," was set to 0.8. In the second one the trust threshold was set to 0.9. A simple look at the distribution of trust scores colored by the aggregated fraud score demonstrates the importance of setting a high trust threshold.

[histogram of trust, colored according to fraud score exp1]
[histogram of trust, colored according to fraud score exp2]

Ben Bowles looked through some of the answers that people gave and concluded that the a threshold 60% would be appropriate to aggregate the votes. For reference, it's interesting to see the dramatic improvement in agreement between the first and second experiments. The following histograms show the distribution of the plurality/majority votes. 

[histogram of % of judgments for majority/plurality exp1]
[histogram of % of judgments for majority/plurality exp2]


Just to wrap up this section, please enjoy the histogram of winner-takes-all aggregated scores. 

[winner take all histogram aggregated scores, exp1]
[winner take all histogram aggregated scores, exp2]

### 1.3 Feature Engineering

The original data set contains very little information about the publisher and advertiser webpages. In fact, it contained only the url. In order to engineer features, I first scraped each webpage. I scraped the titles, the page description/metatags, and the bodies of the websites. For more on the details of this, please see my code on page [insert a link to the page here]. I knew when engineering my features that I wanted to use the CART (classification and regression tree) algorithm because it has decent performance, does not make an assumptions of linearity, and most importantly, is interpretable. If you were to be asked "Why did your algorithm classify this publisher/advertiser page as relevant?" you could actually answer the question clearly for CART, something that cannot be said for other algorithms, and certainly not as well. The features that I engineered were the following: 

1. Cosine similarity on the tf-idf of the title, description, and body

Tf-idf, or term frequency-inverse document frequency, is a matrix with documents as rows and terms as columns. When each term appears in a document, it is given a value which increased based on the number of times that it appears in the document, and which is reduced by the frequency with which the word appears in other documents. This score ranges between 0 and 1, and increases for words which are, generally speaking, more important for the document. The cosine similarity compares two document by multiply their values for each document and adding them all up and ranges between 0 and 1. Documents that are more similar have higher scores, and identical documents have a score of 0. I removed extremely common words using the python Natural Language ToolKit (nltk) package's stopwords for english, and stemmed (removed suffices, pluralities, etc. to find common word roots) again using nltk. 

2. One-, two-, and three-gram similarity

Ngrams are N length sequences of words. I used the ngram packages to calculate similarity (a score of between 0 and 1 with 1 be identical) for seuqences of one, two, or three words in length for the title, description, and the body of the webpages. 

3. Longest common string

The longest common string algorithm calculates the longest substring that two strings have in common. This often returned a nonsense string, so in order to make sure that this longest common substring was meaningful, I tokenized (divided the string into individual words), removed stopwords (common words like 'the' and 'and'), and then ensured that the word in the substring had at least three characters. I then removed the words that appeared too frequently (ex. 'all' 'rights' 'reserved') to be meaningful, and then created a simple binary variable. 

4. Named Entity Recognition (NER)

I used to the Stanford-NER algorithm to extract named entities from the title, description, and body of the webpages. I then converted this back into a string, constructed the tf-idf matrix, and calculated cosine similarity based on the named entities. For the first pass through, I am ony extracting the names. Looking through the webpages makes it clear that most of them are about music, and the Stanford-NER program is not very good with popular music band names, and much better at recognizing people's names. 


### 1.4 Reduction in data set size

You might wonder about how much data was lost by removing the judgments of fraudulent workers and the publisher/ad pairs for which there was not sufficient consensus. 

[table summarizing this for both approaches]

## 2 Training the model 

I divided my remaining into 80% training and 20% test set. Although I wanted to stick with a CART model, I also trained a random forest model [insert parameters of most successful version] and a logistic regression. 

## 3 Testing the model 

There are some basic measures of model performance 

## 4 Pipeline

Pipeline graphic from notebook, but pretty. 


PostgreSQL tables:

1. webpage_data_table = titles, descriptions, body of all urls from exp1
2. webpage_data_table_exp2 = titles, descriptions, body of all urls from exp2

3. worker_table = worker features from exp1
4. worker_table_exp2 = worker features from exp2

5. scores = aggregated scores from exp1
6. scores_exp2 = aggregated scores from exp2

7. signals = extracted features from exp1
8. signals_exp2 = extracted features from exp2



# Insights

My project, as you would hope, is simply chocked full of insights. 

### Quality control 




# A tour of the ratsnest I call my code

You might want to look through my code. I welcome this and I tried to make it relatively organized. The pipeline graphic does the most to help orient to the various pieces. 

# Dictionary: 

1. ground truth
2. score
3. worker 
4. supervised learning
5. unsupervised learning
6. job
7. crowdsource
8. human intelligence task
9. bot
